# Hadoop Streaming assignment 1: Words Rating

The purpose of this task is to create your own WordCount program for Wikipedia dump processing and learn basic concepts of the MapReduce.

In this task you have to find the 7th word by popularity and its quantity in the reverse order (most popular first) in Wikipedia data (`/data/wiki/en_articles_part`).

There are several points for this task:

1) As an output, you have to get the 7th word and its quantity separated by a tab character.

2) You must use the second job to obtain a totally ordered result.

3) Do not forget to redirect all trash and output to /dev/null.

Here you can find the draft of the task main steps. You can use other methods for solution obtaining.

## Step 1. Create mapper and reducer.

<b>Hint:</b>  Demo task contains almost all the necessary pieces to complete this assignment. You may use the demo to implement the first MapReduce Job.

In [1]:
%%writefile mapper1.py

import sys
import re
from imp import reload
if sys.version[0] == '2':
    reload(sys)
    sys.setdefaultencoding("utf-8")
    
for line in sys.stdin:
    try:
        #print ("line", line)
        article_id, text = unicode(line.strip()).split('\t', 1)
    except ValueError as e:
        print("error", e)
        continue
    #text = re.sub("^\W+|\W+$", "", text, flags=re.UNICODE)
    words = re.split("\W*\s+\W*", text, flags=re.UNICODE)
    #print (words)
    for word in words:
        #print (word)
        print >> sys.stderr, "reporter:counter:Wiki stats,Total words,%d" % 1
        #print "%s\t%s" % (word.lower(), 1)
        print ("%s\t%d" % (word.lower(), 1))

Overwriting mapper1.py


In [2]:
%%writefile reducer1.py

# Your code for reducer here.
import sys

current_key = None
word_sum = 0

for line in sys.stdin:
    try:
        key, count = line.strip().split('\t', 1)
        count = int(count)
    except ValueError as e:
        continue
    if current_key != key:
        if current_key:
            print ("%s\t%d" % (current_key, word_sum))
        word_sum = 0
        current_key = key
    word_sum += count

if current_key:
    print ("%s\t%d" % (current_key, word_sum))

Overwriting reducer1.py


In [3]:
# You can use this cell for other experiments: for example, for combiner.

## Step 3. Bash commands

<b> Hint: </b> For printing the exact row you may use basic UNIX commands. For instance, sed/head/tail/... (if you know other commands, you can use them).

To run both jobs, you must use two consecutive yarn-commands. Remember that the input for the second job is the ouput for the first job.

In [4]:
%%bash

OUT_DIR_JOB1="wordsrating_job1_"$(date +"%s%6N")
OUT_DIR_JOB2="wordsrating_job2_"$(date +"%s%6N")

# Code for your first job
# yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar ...
NUM_REDUCERS=8
hdfs dfs -rm -r -skipTrash ${OUT_DIR_JOB1} > /dev/null

yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.name="Streaming wordsRating Job 1" \
    -D mapreduce.job.reduces=${NUM_REDUCERS} \
    -files mapper1.py,reducer1.py \
    -mapper "python mapper1.py" \
    -combiner "python reducer1.py" \
    -reducer "python reducer1.py" \
    -input /data/wiki/en_articles_part \
    -output ${OUT_DIR_JOB1} > /dev/null

# Code for your second job
yarn jar /opt/cloudera/parcels/CDH/lib/hadoop-mapreduce/hadoop-streaming.jar \
    -D mapreduce.job.output.key.comparator.class=org.apache.hadoop.mapreduce.lib.partition.KeyFieldBasedComparator \
    -D mapreduce.job.name="Streaming wordsRating Job 2" \
    -D stream.map.output.field.separator="\t" \
    -D stream.num.map.output.key.fields=2 \
    -D mapreduce.partition.keycomparator.options="-k2,2nr" \
    -D mapreduce.job.reduces=1 \
    -files reducer1.py \
    -mapper /bin/cat \
    -reducer /bin/cat \
    -input ${OUT_DIR_JOB1} \
    -output ${OUT_DIR_JOB2} > /dev/null

# Code for obtaining the results
hdfs dfs -cat ${OUT_DIR_JOB2}/part-00000 | sed -n '7p;8q'
hdfs dfs -rm -r -skipTrash ${OUT_DIR_JOB2}* > /dev/null

packageJobJar: [/tmp/hadoop-unjar7116176628512418307/] [] /tmp/streamjob2027331284089294345.jar tmpDir=null
packageJobJar: [/tmp/hadoop-unjar657501699728138246/] [] /tmp/streamjob3636859616533705165.jar tmpDir=null
is	126420	


rm: `wordsrating_job1_1545448365391146': No such file or directory
18/12/22 03:12:47 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/12/22 03:12:48 INFO client.RMProxy: Connecting to ResourceManager at /0.0.0.0:8032
18/12/22 03:12:48 INFO mapred.FileInputFormat: Total input files to process : 1
18/12/22 03:12:48 INFO mapreduce.JobSubmitter: number of splits:2
18/12/22 03:12:48 INFO mapreduce.JobSubmitter: Submitting tokens for job: job_1545429137258_0014
18/12/22 03:12:48 INFO impl.YarnClientImpl: Submitted application application_1545429137258_0014
18/12/22 03:12:48 INFO mapreduce.Job: The url to track the job: http://3bfb327c519e:8088/proxy/application_1545429137258_0014/
18/12/22 03:12:48 INFO mapreduce.Job: Running job: job_1545429137258_0014
18/12/22 03:12:54 INFO mapreduce.Job: Job job_1545429137258_0014 running in uber mode : false
18/12/22 03:12:54 INFO mapreduce.Job:  map 0% reduce 0%
18/12/22 03:13:09 INFO mapreduce.Job:  map 46% reduce 0%
18/12/22 03:1